In [1]:
import os
import glob
import torch
import random
import pandas as pd
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader, Dataset


In [2]:
# Define relevant variables for the ML task
batch_size = 16
num_classes = 5
learning_rate = 0.001
num_epochs = 50
#  Device will determine whether to run the training on GPU or CPU.
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
data_path = './'
train_file_name = 'train_dataset.csv'
test_file = 'val_dataset.csv'
train_csv_file = os.path.join(data_path,train_file_name)
val_csv_file = os.path.join(data_path,test_file)


In [4]:
# example
data = pd.read_csv(train_csv_file)
n=1
label1 = data.iloc[2,1]
file = data.iloc[n,0]
print('Label is:',label1)
# print('File is:',file)

Label is: 3


In [5]:
class CsvDataset(Dataset):
    def __init__(self, csv_file, transform=None):
        self.data = pd.read_csv(csv_file)
        self.transform = transform

    def __len__(self):
        return len(self.data)
    
    def __getitem__(self,idx):
#         if torch.is_tensor(idx):
#             idx = idx.tolist()
        label= self.data.iloc[idx,1]
        np_file = self.data.iloc[idx,0]
        np_data = np.load(np_file)
        np_data = torch.from_numpy(np_data).float()

        return np_data,label

In [6]:
#######################################################
#                  Create Dataset
#######################################################

train_dataset = CsvDataset(train_csv_file)
train_loader = DataLoader(train_dataset, batch_size)

valid_dataset = CsvDataset(val_csv_file)
valid_loader = DataLoader(valid_dataset,batch_size)

dataiter = iter(train_loader)
files, labels = dataiter.next()
print(files.shape)
print(labels.shape)

torch.Size([16, 3, 38, 114])
torch.Size([16])


In [7]:
class Net(nn.Module):
    def __init__(self, num_classes):
        super(Net, self).__init__()
        self.conv_layer1 = nn.Conv2d(in_channels=3, out_channels=38, kernel_size=3)
        self.conv_layer2 = nn.Conv2d(in_channels=38, out_channels=14, kernel_size=3)
        self.conv_layer3 = nn.Conv2d(in_channels=14, out_channels=4, kernel_size=3)
        
        self.fc1 = nn.Linear(13824, 128)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(128, num_classes)
    
    def forward(self, x):
        out = self.conv_layer1(x)
        out = self.conv_layer2(out)        
        out = self.conv_layer3(out)

        out = out.reshape(out.size(0), -1)
        out = self.fc1(out)
        out = self.relu1(out)
        out = self.fc2(out)
        return out

In [8]:
net = Net(num_classes)

if torch.cuda.is_available():
    net = net.cuda()
    
# print(net)

In [9]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
total_step = len(train_loader)

In [10]:
# net.train(True)

In [11]:
min_valid_loss = np.inf

for epoch in range(num_epochs):  # loop over the dataset multiple times
#     last_loss = 0.
    running_loss = 0.0
    for i, data in enumerate(train_loader):
        # get the inputs; data is a list of [inputs, labels]
        inputs,labels = data
        if torch.cuda.is_available():
            inputs.to(device)
            labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        

        # print statistics
        print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))
        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0
        net.eval()     # Optional when not using Model Specific layer
        with torch.no_grad():
            for data, labels in valid_loader:
                if torch.cuda.is_available():
                    data, labels = data.cuda(), labels.cuda()

                target = net(data)
                loss = criterion(target,labels)
                valid_loss = loss.item() * data.size(0)

            print(f'Epoch {epoch+1} \t\t Training Loss: {running_loss / len(train_loader)} \t\t Validation Loss: {valid_loss / len(valid_loader)}')
            if min_valid_loss > valid_loss:
                print(f'Validation Loss Decreased({min_valid_loss:.6f}--->{valid_loss:.6f}) \t Saving The Model')
                min_valid_loss = valid_loss
                # Saving State Dict
                torch.save(net.state_dict(), 'saved_model.pth')
print('Finished Training')

Epoch [1/50], Loss: 1.5868
Epoch 1 		 Training Loss: 0.39670461416244507 		 Validation Loss: 1.9585633277893066
Validation Loss Decreased(inf--->7.834253) 	 Saving The Model
Epoch [1/50], Loss: 1.6289
Epoch 1 		 Training Loss: 0.8039250075817108 		 Validation Loss: 1.9670853018760681
Epoch [1/50], Loss: 1.6129
Epoch 1 		 Training Loss: 1.2071466147899628 		 Validation Loss: 1.9788016378879547
Epoch [1/50], Loss: 1.5688
Epoch 1 		 Training Loss: 1.599349319934845 		 Validation Loss: 1.9478175044059753
Validation Loss Decreased(7.834253--->7.791270) 	 Saving The Model
Epoch [2/50], Loss: 1.5279
Epoch 2 		 Training Loss: 0.38196665048599243 		 Validation Loss: 1.9194580614566803
Validation Loss Decreased(7.791270--->7.677832) 	 Saving The Model
Epoch [2/50], Loss: 1.5502
Epoch 2 		 Training Loss: 0.7695091664791107 		 Validation Loss: 1.8991950154304504
Validation Loss Decreased(7.677832--->7.596780) 	 Saving The Model
Epoch [2/50], Loss: 1.5524
Epoch 2 		 Training Loss: 1.157606810331344

Epoch [16/50], Loss: 0.0104
Epoch 16 		 Training Loss: 0.002606388181447983 		 Validation Loss: 0.15845540910959244
Epoch [16/50], Loss: 0.1926
Epoch 16 		 Training Loss: 0.0507533997297287 		 Validation Loss: 0.2665049396455288
Epoch [16/50], Loss: 0.0805
Epoch 16 		 Training Loss: 0.07087630778551102 		 Validation Loss: 0.497969314455986
Epoch [16/50], Loss: 0.4470
Epoch 16 		 Training Loss: 0.18263231217861176 		 Validation Loss: 0.1995328813791275
Epoch [17/50], Loss: 0.0091
Epoch 17 		 Training Loss: 0.002273686695843935 		 Validation Loss: 0.08529000915586948
Validation Loss Decreased(0.358371--->0.341160) 	 Saving The Model
Epoch [17/50], Loss: 0.2387
Epoch 17 		 Training Loss: 0.06194594921544194 		 Validation Loss: 0.09867790155112743
Epoch [17/50], Loss: 0.1233
Epoch 17 		 Training Loss: 0.09275877708569169 		 Validation Loss: 0.1682799682021141
Epoch [17/50], Loss: 0.1913
Epoch 17 		 Training Loss: 0.14058405021205544 		 Validation Loss: 0.13374651782214642
Epoch [18/50], Lo

Epoch [34/50], Loss: 0.0014
Epoch 34 		 Training Loss: 0.00034523732028901577 		 Validation Loss: 0.09019484743475914
Epoch [34/50], Loss: 0.0094
Epoch 34 		 Training Loss: 0.0027045956812798977 		 Validation Loss: 0.09489945136010647
Epoch [34/50], Loss: 0.0116
Epoch 34 		 Training Loss: 0.005604124395176768 		 Validation Loss: 0.10838925838470459
Epoch [34/50], Loss: 0.0165
Epoch 34 		 Training Loss: 0.009735917439684272 		 Validation Loss: 0.09919853881001472
Epoch [35/50], Loss: 0.0013
Epoch 35 		 Training Loss: 0.0003220565558876842 		 Validation Loss: 0.09155328385531902
Epoch [35/50], Loss: 0.0086
Epoch 35 		 Training Loss: 0.0024625536461826414 		 Validation Loss: 0.09536532685160637
Epoch [35/50], Loss: 0.0105
Epoch 35 		 Training Loss: 0.005099169531604275 		 Validation Loss: 0.10751191526651382
Epoch [35/50], Loss: 0.0148
Epoch 35 		 Training Loss: 0.0088089493510779 		 Validation Loss: 0.0988423079252243
Epoch [36/50], Loss: 0.0012
Epoch 36 		 Training Loss: 0.0003021263983

In [12]:
print(outputs.shape)

torch.Size([14, 5])


In [13]:
with torch.no_grad():
    correct = 0
    total = 0
    for data, labels in valid_loader:
        data = data.to(device)
        labels = labels.to(device)
        outputs = net(data)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    print('Accuracy of the network on the data: {} %'.format( 100 * correct / total))



Accuracy of the network on the data: 90.56603773584905 %
